Train a classifier that can predict the patient condition based on the drug review.

In [ ]:
!pip install 'transformers[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 24.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.0 MB/s eta 0:00:00


In [ ]:
# Load the datasets
from datasets import load_dataset

data_files = {
    'train': '/content/drive/MyDrive/Colab Notebooks/NLP/5. THE DATASETS LIBRARY/DrugDatasetClassifier/data/drug-reviews-train.csv',
    'validation': '/content/drive/MyDrive/Colab Notebooks/NLP/5. THE DATASETS LIBRARY/DrugDatasetClassifier/data/drug-reviews-validation.csv',
    'test': '/content/drive/MyDrive/Colab Notebooks/NLP/5. THE DATASETS LIBRARY/DrugDatasetClassifier/data/drug-reviews-test.csv'
}

drug_datasets = load_dataset('csv', data_files=data_files)
drug_datasets

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-134572b29369406d/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 147318
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 16369
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 18188
    })
})

In [ ]:
# Number of unique labels
unique_labels = list(
    set(
        drug_datasets['train'].unique('condition') +
        drug_datasets['validation'].unique('condition') +
        drug_datasets['test'].unique('condition')
    )
)

print(len(unique_labels))

332


In [ ]:
# Number of unique drugs
len(list(
    set(
        drug_datasets['train'].unique('drugName') +
        drug_datasets['validation'].unique('drugName') +
        drug_datasets['test'].unique('drugName')
    )
))

2948

In [ ]:
# Inspect a review
drug_datasets['train']['review'][0]

'"I\'ve always been a sufferer of chronic constipation.  Thought that Amitiza would be my miracle pill. Here are my pros and cons:\r\r\nPro - When I actually go, I REALLY go. I\'m not feeling left bloated and cramped. \r\r\n\r\r\nCon - I\'m usually nauseous for about an hour after taking my pills. I\'m not able to go on my own. I have to use a Fleet to get things going."'

In [ ]:
drugs = drug_datasets.remove_columns(['patient_id', 'drugName', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'])
drugs

DatasetDict({
    train: Dataset({
        features: ['condition', 'review'],
        num_rows: 147318
    })
    validation: Dataset({
        features: ['condition', 'review'],
        num_rows: 16369
    })
    test: Dataset({
        features: ['condition', 'review'],
        num_rows: 18188
    })
})

In [ ]:
# Tokenization
from transformers import AutoTokenizer

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenization_function(inp):
    return tokenizer(inp['review'], truncation=True)

tokenized_dataset = drugs.map(tokenization_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/147318 [00:00<?, ? examples/s]

Map:   0%|          | 0/16369 [00:00<?, ? examples/s]

Map:   0%|          | 0/18188 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['condition', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 147318
    })
    validation: Dataset({
        features: ['condition', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16369
    })
    test: Dataset({
        features: ['condition', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18188
    })
})

In [ ]:
# Remove 'review' from `tokenized_dataset` since it is no longer required for the model.
# Rename conditions to labels
tokenized_dataset = tokenized_dataset.remove_columns(['review'])
tokenized_dataset = tokenized_dataset.rename_column('condition', 'labels')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 147318
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16369
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18188
    })
})

In [ ]:
# Dynamic padding with datacollator
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Training

In [ ]:
# Define TrainingArguments
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir='checkpoints', evaluation_strategy='epoch')

In [ ]:
# Define your model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=332)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
!pip install evaluate
import numpy as np
import evaluate

def compute_metrics(eval_preds):
    metrics = evaluate.combine([
        evaluate.load('accuracy'),
        evaluate.load('f1', average='macro'),
        evaluate.load('precision', average='macro'),
        evaluate.load('recall', average='macro'),
    ])
    logits, labels = evaluate_preds
    predictions = np.argmax(logits, axis=-1)
    return metrics.compute(predictions=predictions, references=labels)

In [ ]:
!pip install evaluate
import numpy as np
import evaluate

def compute_metrics(eval_preds):
    metrics = evaluate.load(['accuracy', 'precision', 'f1', 'recall'], average='macro')
    logits, labels = evaluate_preds
    predictions = np.argmax(logits, axis=-1)
    return metrics.compute(predictions=predictions, references=labels)

In [ ]:
# Define compute metrics

!pip install evaluate
import evaluate
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load('accuracy')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.2 MB/s eta 0:00:00


In [ ]:
# Define Trainer object
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the network
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1943 in _inner_training_loop     │
│                                                                                                  │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│   1942 │   │   │   │   │   and not is_torch_tpu_available()                                      │
│ ❱ 1943 │   │   │   │   │   and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))          │
│   1944 │   │   │   │   ):                                                                        │
│   1945 │   │   │   │   │   # if loss is nan or inf simply add the average of previous logged lo  │
│   1946 │   │   │   │   │   tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,